In [114]:
import numpy as np
import pandas as pd
import random
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
from collections import defaultdict
from IPython.display import display_html
from itertools import chain, cycle
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import *
from src.util import *
from src import config
from src.visualize import *
from src.decode import *
pd.set_option('display.float_format', lambda x: '%.04f' % x)
gezi.set_pandas_widder()

In [115]:
# !ls ../working/offline/47

In [116]:
from src.ensemble_conf import v, mns
ic(v, mns)

ic| 461123328.py:2 in <module>
    v: 60
    mns: ['deberta-v3.start.len1024.rnn_bi',
          'deberta-v3.end.len1024.seq_encoder-0',
          'deberta-v3.start.stride-256.seq_encoder-0',
          'deberta-v3.start.nwemb-0.mark_end-0',
          'deberta-v3.se',
          'bart.start.run2',
          'deberta-v3.start.len1024.stride-256.seq_encoder-0',
          'deberta-xlarge.start',
          'deberta-v3.mid.len1024',
          'deberta-xlarge.end',
          'deberta-v3.start.len1024.stride-256',
          'deberta.start',
          'longformer.start.len1536',
          'deberta-v3.start.len1536',
          'roberta.start.nwemb-0',
          'deberta-v3.se2']


(60,
 ['deberta-v3.start.len1024.rnn_bi',
  'deberta-v3.end.len1024.seq_encoder-0',
  'deberta-v3.start.stride-256.seq_encoder-0',
  'deberta-v3.start.nwemb-0.mark_end-0',
  'deberta-v3.se',
  'bart.start.run2',
  'deberta-v3.start.len1024.stride-256.seq_encoder-0',
  'deberta-xlarge.start',
  'deberta-v3.mid.len1024',
  'deberta-xlarge.end',
  'deberta-v3.start.len1024.stride-256',
  'deberta.start',
  'longformer.start.len1536',
  'deberta-v3.start.len1536',
  'roberta.start.nwemb-0',
  'deberta-v3.se2'])

In [117]:
info = gezi.load(f'../working/offline/{v}/valid.pkl')
info = gezi.batch2list(info)
info = gezi.sort_list_byid(info)

In [118]:
folds = 5
infos = [info]

for mn in tqdm(mns):
  info_ = [gezi.batch2list(gezi.load(f'../working/offline/{v}/{fold}/{mn}/valid.pkl')) for fold in range(folds)]
  info_ = list(chain.from_iterable(info_))
  info_ = gezi.sort_list_byid(info_)
  infos.append(info_)

  0%|          | 0/16 [00:00<?, ?it/s]

In [119]:
# ['deberta-v3.start.lovasz_loss_rate-1.',
#           'deberta-v3.end.lovasz_loss_rate-1.',
#           'deberta-v3.se.lovasz_loss_rate-1.',
#           'deberta-v3.mid.lovasz_loss_rate-1.',
#           'deberta.start.lovasz_loss_rate-1.',
#           'deberta.end.lovasz_loss_rate-1.',
#           'deberta.se.lovasz_loss_rate-1.',
#           'deberta.mid.lovasz_loss_rate-1.',
#           'deberta-v3.se2.lovasz_loss_rate-1.',
#           'deberta-v3.se3.lovasz_loss_rate-1.',
#           'deberta-v3.se4.lovasz_loss_rate-1.',
#           'bart.start.x',
#           'deberta-v3.start.mui-end-mid.lovasz_loss_rate-1',
#           'deberta.start.mui-end-mid.lovasz_loss_rate-1',
#           'longformer.start.len1536.seq_encoder.lovasz_loss_rate-1',
#           'bird.start.len1536.seq_encoder.bs-8.half_lr']

In [120]:
dfs = []
fold = 0
for i, mn in tqdm(enumerate(mns), total=len(mns)):
  d = pd.read_csv(f'../working/offline/{v}/{fold}/{mn}/valid_pred.csv')
  # ic(i, len(d))
  d['model'] = i + 2
  d['model_name'] = mn
  dfs.append(d)

  0%|          | 0/16 [00:00<?, ?it/s]

In [121]:
df_gt = pd.read_csv(f'../working/offline/{v}/valid_gt.csv')

In [122]:
df_gt = df_gt[df_gt.kfold==0]

In [123]:
d = pd.read_csv(f'../working/offline/{v}/valid_pred.csv')
d['model'] = 1
d['model_name'] = 'ensemble'
d = d[d.id.isin(df_gt.id)]
dfs.insert(0, d)

In [124]:
calc_metrics(df_gt, dfs[0])

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

{'f1/Overall': 0.7239920553288934,
 'f1/Claim': 0.6933115823817292,
 'acc/Claim': 0.7139857202855943,
 'recall/Claim': 0.6738010305192231,
 'f1/Evidence': 0.7793395427603725,
 'acc/Evidence': 0.8008120649651972,
 'recall/Evidence': 0.7589884551951622,
 'f1/Position': 0.7421009098428454,
 'acc/Position': 0.7521797451374916,
 'recall/Position': 0.732288605941887,
 'f1/Concluding Statement': 0.8761518614080354,
 'acc/Concluding Statement': 0.8627949183303085,
 'recall/Concluding Statement': 0.8899288655934107,
 'f1/Lead': 0.8457142857142858,
 'acc/Lead': 0.8409090909090909,
 'recall/Lead': 0.8505747126436781,
 'f1/Counterclaim': 0.5990295544772828,
 'acc/Counterclaim': 0.6057091882247992,
 'recall/Counterclaim': 0.5924956369982548,
 'f1/Rebuttal': 0.5322966507177034,
 'acc/Rebuttal': 0.5507425742574258,
 'recall/Rebuttal': 0.5150462962962963}

In [125]:
info[0].keys()

dict_keys(['id', 'pred', 'word_ids', 'num_words', 'label', 'start', 'start_logits', 'start_probs', 'probs', 'preds'])

In [126]:
df_l2 = pd.read_csv(f'../working/offline/60/{fold}/l2/valid.csv')

In [127]:
df_l2 = df_l2.sort_values(['id', 'pred'], ascending=[True, False])
df_l2 

,Unnamed: 0,id,model,fold,num_gts,num_words,gt,pred
8463,8463,0000D23A521A,14,0,8,251,0.6286,0.7427
7466,7466,0000D23A521A,8,0,8,251,0.3429,0.7245
5992,5992,0000D23A521A,7,0,8,251,0.3810,0.7237
7247,7247,0000D23A521A,15,0,8,251,0.4857,0.6961
8058,8058,0000D23A521A,4,0,8,251,0.3810,0.6939
...,...,...,...,...,...,...,...,...
33583,33583,FFF1442D6698,16,0,10,651,0.6667,0.5563
33657,33657,FFF1442D6698,9,0,10,651,0.6190,0.5553
35476,35476,FFF1442D6698,8,0,10,651,0.5000,0.4922
33225,33225,FFF1442D6698,6,0,10,651,0.4952,0.4230


In [128]:
df_l2[df_l2.model==1]['gt'].mean()

0.713975917062702

In [129]:
df_ = df_l2.groupby('id').first().reset_index()
df_

,id,Unnamed: 0,model,fold,num_gts,num_words,gt,pred
0,0000D23A521A,8463,14,0,8,251,0.6286,0.7427
1,001A03E06F3C,1404,8,0,7,381,0.7333,0.8778
2,004BE23B05BF,29405,15,0,10,513,0.9333,0.8774
3,0054850878E3,50043,14,0,9,475,0.8800,0.7845
4,00852F390697,49853,15,0,11,502,0.4722,0.7831
...,...,...,...,...,...,...,...,...
3089,FF9114183593,25806,14,0,11,329,0.6000,0.7604
3090,FF9DB9D43F72,33317,15,0,7,567,0.5600,0.8630
3091,FFA381E58FC6,36395,16,0,5,173,0.2917,0.7437
3092,FFC43F453EF6,17395,8,0,11,192,0.3333,0.5907


In [130]:
dm1 = df_l2[df_l2.model==1]
dm1

,Unnamed: 0,id,model,fold,num_gts,num_words,gt,pred
6979,6979,0000D23A521A,1,0,8,251,0.4857,0.6891
1047,1047,001A03E06F3C,1,0,7,381,0.7333,0.8559
29166,29166,004BE23B05BF,1,0,10,513,0.9333,0.8558
48592,48592,0054850878E3,1,0,9,475,0.4800,0.5820
48706,48706,00852F390697,1,0,11,502,0.4841,0.7309
...,...,...,...,...,...,...,...,...
25239,25239,FF9114183593,1,0,11,329,0.8857,0.6423
35256,35256,FF9DB9D43F72,1,0,7,567,0.6000,0.8349
35969,35969,FFA381E58FC6,1,0,5,173,0.6167,0.5202
17151,17151,FFC43F453EF6,1,0,11,192,0.4667,0.5428


In [131]:
df_['gt'].mean()

0.7065657001011635

In [132]:
ids, classes, preds = [], [], []
count = 0
for row in tqdm(df_.itertuples(), total=len(df_)):
  count += 1
  pred = dm1[dm1.id==row.id].pred.values[0]
  model = row.model - 1 if row.pred - pred > 0.2 else 0
  # model = 0
  d = dfs[model].copy()
  d = d[d.id== row.id]
  if len(d):
    ids.extend(list(d.id.values))
    classes.extend(list(d['class'].values))
    preds.extend(list(d['predictionstring'].values))
  # ic(row.id, calc_metrics(df_gt[df_gt.id==row.id], d, f1_only=True))
  # if count == 5:
  #   break

m = {
  'id': ids,
  'class': classes,
  'predictionstring': preds,
}
d2 = pd.DataFrame(m)
calc_metrics(df_gt, d2)

  0%|          | 0/3094 [00:00<?, ?it/s]

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

{'f1/Overall': 0.7225745842711451,
 'f1/Claim': 0.6919479463196422,
 'acc/Claim': 0.7104384133611691,
 'recall/Claim': 0.6743955608402695,
 'f1/Evidence': 0.7784992377618429,
 'acc/Evidence': 0.8001392757660167,
 'recall/Evidence': 0.7579989004947774,
 'f1/Position': 0.7393520802499589,
 'acc/Position': 0.7448641484426772,
 'recall/Position': 0.7339209924910218,
 'f1/Concluding Statement': 0.8749769839808507,
 'acc/Concluding Statement': 0.8608695652173913,
 'recall/Concluding Statement': 0.8895544739797828,
 'f1/Lead': 0.8417618270799347,
 'acc/Lead': 0.8363047001620746,
 'recall/Lead': 0.8472906403940886,
 'f1/Counterclaim': 0.5998233215547704,
 'acc/Counterclaim': 0.6073345259391771,
 'recall/Counterclaim': 0.5924956369982548,
 'f1/Rebuttal': 0.5316606929510155,
 'acc/Rebuttal': 0.5493827160493827,
 'recall/Rebuttal': 0.5150462962962963}

In [133]:
calc_metrics(df_gt, dfs[0])

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

{'f1/Overall': 0.7239920553288934,
 'f1/Claim': 0.6933115823817292,
 'acc/Claim': 0.7139857202855943,
 'recall/Claim': 0.6738010305192231,
 'f1/Evidence': 0.7793395427603725,
 'acc/Evidence': 0.8008120649651972,
 'recall/Evidence': 0.7589884551951622,
 'f1/Position': 0.7421009098428454,
 'acc/Position': 0.7521797451374916,
 'recall/Position': 0.732288605941887,
 'f1/Concluding Statement': 0.8761518614080354,
 'acc/Concluding Statement': 0.8627949183303085,
 'recall/Concluding Statement': 0.8899288655934107,
 'f1/Lead': 0.8457142857142858,
 'acc/Lead': 0.8409090909090909,
 'recall/Lead': 0.8505747126436781,
 'f1/Counterclaim': 0.5990295544772828,
 'acc/Counterclaim': 0.6057091882247992,
 'recall/Counterclaim': 0.5924956369982548,
 'f1/Rebuttal': 0.5322966507177034,
 'acc/Rebuttal': 0.5507425742574258,
 'recall/Rebuttal': 0.5150462962962963}

In [134]:
len(dfs[0])

27649

In [135]:
id = 'EC8FCCDB698A'
total = len(info)
x = [a for a in info if a['id'] == id][0]

In [136]:
gezi.init_flags()

In [137]:
 res_list = decode(x)

In [138]:
d_text = pd.read_feather('../input/feedback-prize-2021/texts.fea')

In [139]:
text = d_text[d_text.id==x['id']].text.values[0]

In [140]:
text, len(text.split())

('Venus is worth studying because when they study it they called it an "Evening Star" also a "twin" of earth. It\'s the closest planet to earth, It is the closest in distence as well. Venus also has a 97% carbon diocide blanket, but on the surface its 800\n\ndegrees Fahrenheight. Since the conditions of the planet are like that its way to extreme for humans to live there. Venus is also has the hottest suface temperature in our solar system, even thou Mercury is closest to the sun Venus is till hotter. It has eruputing volcanoes, powerful earthquakes and frequesnt lightning strikes that happen on the surface of the planet. Scientist think that long ago Venus was like Earth, but even now it still has some of the features that Earth has. It has the rocky sediment, valleys, mountains, craters. NASA is working to keep studying Venus. They have done fom silicon carbide tests that are simulations of the chaos of Venus\'s surface. One of the things is that they are still striving to meet the c

In [141]:
def get_ents(res_list, starts):
  ents = []
  for res in res_list:
    ents.append({
      'start': starts[res['start']],
      'end': starts[res['end']],
      'label': id2dis[res['cls']]
    })
  return ents  
    
def visualize(text, res_list, res_list2=None):
  
  text = text.replace('[BR]', '')
  words = text.split()
  text = ' '.join(words)
  starts = [0]
  for i in range(1, len(words)):
    start = starts[i - 1] + len(words[i - 1]) + 1
    starts.append(start)
  start = starts[len(words) - 1] + len(words[-1])
  starts.append(start)
    
  ents = get_ents(res_list, starts)
  doc = {
    'text': text,
    'ents': ents
  }
  
  colors = {
      'Lead': '#EE11D0',
      'Position': '#AB4DE1',
      'Claim': '#1EDE71',
      'Evidence': '#33FAFA',
      'Counterclaim': '#4253C1',
      'Concluding Statement': 'yellow',
      'Rebuttal': 'red'
  }
  options = {
      "ents": ALL_CLASSES,
      "colors": colors
  }
  
  spacy.displacy.render(doc,
                        style="ent",
                        options=options,
                        manual=True, 
                        jupyter=True)
  if res_list2 is not None:
    ents = get_ents(res_list2, starts)
    doc = {
      'text': text,
      'ents': ents
    }
    spacy.displacy.render(doc,
                      style="ent",
                      options=options,
                      manual=True, 
                      jupyter=True)
    # display_side_by_side([(r1, "label"), (r2, "pred")])

In [142]:
x['id']

'EC8FCCDB698A'

In [143]:
visualize(text, res_list)

In [144]:
label = pd.read_feather('../input/feedback-prize-2021/train_en.fea')

In [145]:
d = label[label.id == x['id']]

In [146]:
res_list2 = gezi.batch2list({'cls': d.para_type.values, 'start': d.start.values, 'end': d.end.values})

In [147]:
visualize(text, res_list, res_list2)

In [148]:
def res2dict(res_list):
  m = {}
  for cls in CLASSES:
    m[cls] = []
  for res in res_list:
    if not res['cls']:
      continue
    m[id2dis[res['cls']]].append([res['start'], res['end']])
  return m

In [149]:
m1 = res2dict(res_list)

In [150]:
m2 = res2dict(res_list2)

In [151]:
res = essay_f1(m2, m1)
res

0.5777777777777778

In [152]:
res

0.5777777777777778

In [153]:
id2idx = {
  x['id']: i for i, x in enumerate(info)
}

In [154]:
def visualize_essay(i, info, label):
  x = {}
  
  if isinstance(i, str):
    id = i
    i = id2idx[id]
  x = info[i]
  id = x['id']
  ic(id)
  
  d = label[label.id == id]
  res_list1 = gezi.batch2list({'cls': d.para_type.values, 'start': d.start.values, 'end': d.end.values})
  m1 = res2dict(res_list1)
  res_list2 = decode(x)
  m2 = res2dict(res_list2)
  res = essay_f1(m1, m2, return_dict=True)
  gezi.pprint(res)
  text = d.text.values[0]
  visualize(text, res_list1, res_list2)

vis = lambda i: visualize_essay(i, info, label)

In [155]:
def visualize_ensembles(i, infos, label):
  x = {}
  
  if isinstance(i, str):
    id = i
    i = id2idx[id]
  else:
    id = infos[0][i]['id']
  ic(id)
  
  d = label[label.id == id]
  res_list1 = gezi.batch2list({'cls': d.para_type.values, 'start': d.start.values, 'end': d.end.values})
  m1 = res2dict(res_list1)
  text = d.text.values[0]
  print('gt')
  visualize(text, res_list1)
  
  scores = []
  for j, info in enumerate(infos):
    x = info[i]
    res_list2 = decode(x)
    
    m2 = res2dict(res_list2)
    res = essay_f1(m1, m2, return_dict=True)
    model_name = 'ensemble' if j == 0 else mns[j - 1]
    print(j, model_name)
    # if model_name == 'deberta-xlarge.end':
    #   ic(res_list2)
    gezi.pprint(res)
    visualize(text, res_list2)
    scores.append(res['Overall'])
  ic(scores[0], max(scores[1:]))

vis2 = lambda i: visualize_ensembles(i, infos, label)

In [156]:
x = gezi.load('../working/offline/60/3/deberta-xlarge.end/valid.pkl')

In [157]:
x = [a for a in gezi.batch2list(x) if a['id'] == 'BE01ACCDF251'][0]

In [158]:
x['preds']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [159]:
decode(x)

[{'cls': 2,
  'len': 122,
  'start': 0,
  'end': 122,
  'token_prob': array([3.50799310e+00, 4.89055789e-01, 8.71396017e+01, 5.54073848e-02,
         9.28910569e-03, 6.69964642e-01, 6.69257521e-02, 6.17625526e-02]),
  'sep_prob': 0.873935920684122,
  'sep_logits': array([-1.11180115,  0.82441556]),
  'token_logits': array([ 1.04139264, -0.67194138,  3.48490897, -2.18222295, -3.46036848,
         -0.26826103, -2.05055349, -2.10491681])},
 {'cls': 2,
  'len': 119,
  'start': 122,
  'end': 241,
  'token_prob': array([1.13219133e+01, 7.87517654e+00, 9.96703020e+01, 4.49977164e-02,
         8.14921436e-03, 1.86439250e-02, 4.20348464e-02, 1.87824651e-02]),
  'sep_prob': 0.8539448785029286,
  'sep_logits': array([-1.01913142,  0.74675113]),
  'token_logits': array([ 2.31201673,  1.16488428,  4.68507456, -4.13934474, -4.86303674,
         -3.96681051, -3.15252675, -4.01350599])},
 {'cls': 2,
  'len': 83,
  'start': 241,
  'end': 324,
  'token_prob': array([1.06589845e+01, 3.66945185e+00, 6.862

In [160]:
x['pred']

array([[-0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.        , -0.        , -0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.        , -0.        , -0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.76335281, -0.31800193, -0.61968827, ..., -3.94235253,
        -4.10880566, -2.67339516],
       [ 0.90769863, -0.19386414, -0.49647379, ..., -3.96009588,
        -4.297575  , -2.93457031],
       [ 0.59521091, -0.24414374, -0.68975943, ..., -3.69724941,
        -4.03856468, -2.59742618]])

In [162]:
visualize(label[label.id == 'BE01ACCDF251'].text.values[0], decode(x))

In [163]:
vis2(300)

ic| 2987334729.py:9 in visualize_ensembles()- id: '052136F20DD1'


gt


0 ensemble

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |          1 |          0 |                      1 |      1 |              1 |          1 |    0.7143 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |          1 |          0 |                      1 |      1 |              1 |          1 |    0.7143 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |          1 |          0 |                      1 |      1 |              1 |          1 |    0.7143 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.5 |     0.6667 |                      1 |      1 |              1 |     0.6667 |    0.6905 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |          1 |          0 |                      1 |      1 |              1 |          1 |    0.7143 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.5 |          0 |                      1 |      1 |              1 |     0.6667 |    0.5952 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.5 |          0 |                      1 |      1 |              1 |     0.6667 |    0.5952 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.7142857142857143
    max(scores[1:]): 0.7380952380952381


In [164]:
vis(1000)

ic| 2049877657.py:9 in visualize_essay()- id: '1099C84A1A86'



 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|  0.6667 |     0.8333 |          1 |      1 |     0.875 |
+---------+------------+------------+--------+-----------+


In [165]:
vis('BE01ACCDF251')

ic| 2049877657.py:9 in visualize_essay()- id: 'BE01ACCDF251'



 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          0 |                      0 |    0.3333 |
+------------+------------+------------------------+-----------+


In [166]:
vis2('BE01ACCDF251')

ic| 2987334729.py:9 in visualize_ensembles()- id: 'BE01ACCDF251'


gt


0 ensemble

 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          0 |                      0 |    0.3333 |
+------------+------------+------------------------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          0 |                      0 |    0.3333 |
+------------+------------+------------------------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +------------+------------+------------------------+--------+-----------+
|   Evidence |   Position |   Concluding Statement |   Lead |   Overall |
|------------+------------+------------------------+--------+-----------|
|       0.75 |          0 |                      0 |      0 |    0.1875 |
+------------+------------+------------------------+--------+-----------+


...

13 longformer.start.len1536

 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          0 |                      1 |    0.6667 |
+------------+------------+------------------------+-----------+


14 deberta-v3.start.len1536

 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          0 |                      0 |    0.3333 |
+------------+------------+------------------------+-----------+


15 roberta.start.nwemb-0

 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          0 |                      0 |    0.3333 |
+------------+------------+------------------------+-----------+


16 deberta-v3.se2

 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          1 |                      1 |         1 |
+------------+------------+------------------------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.3333333333333333
    max(scores[1:]): 1.0


In [167]:
# label[label.id=='BE01ACCDF251']

In [168]:
label[label.id=='BE01ACCDF251'].text.values[0]

"According to the Articles fellow citizens about the advantages of limiting car usage are going out for controll because the article indacties 70 percemt of Vauban's familes do not own cars, and 57 percant sold a car to move here. Also the Diesels make up 67 percent of vehicles in france , compared to a 5.3 percant average of the diesel engimes in the rest of western Europe , according to reuters. Still however indicates The goal is to promote alternative transportation and reduce smog and the Violators faced $25 fines. The Article also states As April 2013, the number of miles driven per person was nearly 9 percent below th peak and equal to where the country was in januart 1995. [BR] [BR] One Reason, Fellow citizens about the advantages of limiting car usage are going out of controll because in the article Ingerman Suburb,Life Goes On Without Cars states Automoblies are the linchpin of suburbs, where middle- class familes From chicago to shanghai tens to make their homes and that , e

In [169]:
ori = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [170]:
ori[ori.id=='BE01ACCDF251']

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
11821,BE01ACCDF251,1622067810486.0000,48.0000,111.0000,the advantages of limiting car usage are going out for controll,Position,Position 1,7 8 9 10 11 12 13 14 15 16 17
11822,BE01ACCDF251,1622067825243.0000,120.0000,689.0000,"the article indacties 70 percemt of Vauban's familes do not own cars, and 57 percant sold a car to move here. Also the Diesels make up 67 percent of vehicles in france , compared to a 5.3 percant average of the diesel engimes in the rest of western Europe , according to reuters. Still however indicates The goal is to promote alternative transportation and reduce smog and the Violators faced $25 fines. The Article also states As April 2013, the number of miles driven per person was nearly 9 percent below th peak and equal to where the country was in januart 1995.",Evidence,Evidence 1,19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
11823,BE01ACCDF251,1622067791674.0000,703.0000,1406.0000,"Fellow citizens about the advantages of limiting car usage are going out of controll because in the article Ingerman Suburb,Life Goes On Without Cars states Automoblies are the linchpin of suburbs, where middle- class familes From chicago to shanghai tens to make their homes and that , experts say, is a huge impediment to current efforts to drastically reduce greenhouse gas emissions from tailpies for the passenger car are responsibe for 12 percent of greenhouse gas emissions in europe and up to 50 percent in sime car intensive areas in the united states. But in paris bans driving dur to smog takes about while exceptions were made for plug in cars,hybrids, and carrying three or more passengers.",Evidence,Evidence 2,124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240
11824,BE01ACCDF251,1622067782316.0000,1422.0000,1927.0000,",fellow citizens about the advantages of limiting car usage are going out of control because it incates President obamas ambitions goals to curb the united staes greenhouse gas emissons, unveiled lastweek, will get a fortuitios assist from an incipient shoft in american behavior also recent studies suggest that americans are buying fewer cars, driving less and getting fewer licenses as each years goes by. the days without cars is part of an improvement campaign that began in bogotoa in the mid 1990s.",Evidence,Evidence 3,243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
11825,BE01ACCDF251,1622067763915.0000,1936.0000,2385.0000,",Fellow citizens about the advantages of limiting car usage are going out of control because parks and spotys center also have bloomed throughout the city, uneven , pitted sidewalk have been replaced by broad, smoth sidewalks, rush hours restrictions have dramatically cut traffic and new testaurants and upscale shopping districts have cropped up and it counties that cold nights and warm days caused the warmer layer of air to trap car emissions.",Evidence,Evidence 4,325 326 327 328 329 330 331 332 333 334 335 33

In [171]:
label[label.id=='BE01ACCDF251'].text.values[0].replace('[BR]', '\n').split()[241:243]

['Another', 'Reason,fellow']

In [172]:
label[label.id=='BE01ACCDF251'].text.values[0].replace('[BR]', '\n').split()[243:325]

['citizens',
 'about',
 'the',
 'advantages',
 'of',
 'limiting',
 'car',
 'usage',
 'are',
 'going',
 'out',
 'of',
 'control',
 'because',
 'it',
 'incates',
 'President',
 'obamas',
 'ambitions',
 'goals',
 'to',
 'curb',
 'the',
 'united',
 'staes',
 'greenhouse',
 'gas',
 'emissons,',
 'unveiled',
 'lastweek,',
 'will',
 'get',
 'a',
 'fortuitios',
 'assist',
 'from',
 'an',
 'incipient',
 'shoft',
 'in',
 'american',
 'behavior',
 'also',
 'recent',
 'studies',
 'suggest',
 'that',
 'americans',
 'are',
 'buying',
 'fewer',
 'cars,',
 'driving',
 'less',
 'and',
 'getting',
 'fewer',
 'licenses',
 'as',
 'each',
 'years',
 'goes',
 'by.',
 'the',
 'days',
 'without',
 'cars',
 'is',
 'part',
 'of',
 'an',
 'improvement',
 'campaign',
 'that',
 'began',
 'in',
 'bogotoa',
 'in',
 'the',
 'mid',
 '1990s.',
 'However,Fellow']

In [173]:
vis(3)

ic| 2049877657.py:9 in visualize_essay()- id: '001552828BD0'



 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |     0.2857 |          1 |      0 |    0.3214 |
+---------+------------+------------+--------+-----------+


In [174]:
vis2(3)

ic| 2987334729.py:9 in visualize_ensembles()- id: '001552828BD0'


gt


0 ensemble

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |     0.2857 |          1 |      0 |    0.3214 |
+---------+------------+------------+--------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |     0.2857 |          1 |      0 |    0.3214 |
+---------+------------+------------+--------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |        0.5 |          1 |      1 |     0.625 |
+---------+------------+------------+--------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+------------------------+--------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Overall |
|---------+------------+------------+------------------------+--------+-----------|
|       0 |        0.5 |          0 |                      0 |      1 |       0.3 |
+---------+------------+------------+------------------------+--------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |       0.25 |          1 | 0.6667 |    0.4792 |
+---------+------------+------------+--------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |       0.25 |          0 |      1 |    0.3125 |
+---------+------------+------------+--------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|     0.4 |          1 |          1 |      1 |      0.85 |
+---------+------------+------------+--------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.3214285714285714
    max(scores[1:]): 0.85


In [175]:
vis('DBF7EB6A9E02')

ic| 2049877657.py:9 in visualize_essay()- id: 'DBF7EB6A9E02'



 +---------+------------+------------+------------------------+--------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Overall |
|---------+------------+------------+------------------------+--------+-----------|
|  0.4444 |          1 |          1 |                      1 |      0 |    0.6889 |
+---------+------------+------------+------------------------+--------+-----------+


In [176]:
vis('4E1C636ABEAD')

ic| 2049877657.py:9 in visualize_essay()- id: '4E1C636ABEAD'



 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|     0.8 |          1 |          0 |                      1 |       0.7 |
+---------+------------+------------+------------------------+-----------+


In [177]:
# label[label.id=='4E1C636ABEAD']

In [178]:
ori[ori.id=='4E1C636ABEAD']

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
1525,4E1C636ABEAD,1622992466917.0000,126.0000,369.0000,"People should not be able to use cell phones while operating a vehicle.\n\nThere are several reasons why and I will be discussing two of them. Cell phones are a convience to our society, but there should be a time and place for them to be used.\n",Position,Position 1,22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
1526,4E1C636ABEAD,1622992280991.0000,370.0000,478.0000,"First, cell phones are a benefit and allows everyone to have access to a telephone at all times of the day.\n",Claim,Claim 1,69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90
1527,4E1C636ABEAD,1622992426147.0000,479.0000,907.0000,"Kids, teenagers, adults, even grandparents have cell phones.\n\nAs you pay attention to people that you pass by whether it's in traffic or walking, there is a cell phone in toat.\n\nWhile riding a bike, people are on the phone.\n\nWhile driving cars, people are on the phone.\n\nWhile sitting waiting for a bus, people are on the phone.\n\nEven while people are walking they are on the phone.\n\nDo they get any break?\n\nThe phone that is?\n\n",Evidence,Evidence 1,90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
1528,4E1C636ABEAD,1622992346700.0000,907.0000,1043.0000,With all of this phone use where is the focus of the individual?\n\nIndividual focus does not exist when everyone is on their cell phone.\n,Claim,Claim 2,168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193
1529,4E1C636ABEAD,1622992371128.0000,1044.0000,1618.0000,"Whether they are looking at the weather, SnapChat, Facebook, or Instagram, they are still not focused on their surroundings.\n\nWith the lack of focus there have been more accidents and deaths.\n\nAccidents occur when you are distracted and death can easily follow.\n\nWhile driving the attention needs to be on the road.\n\nThere have been documentaries where those that have lost loved ones due to inappropriate cell phone use read the last message that was sent or the last words that they user said.\n\nMany times that call or message wasn't that important and could have waited.\n",Evidence,Evidence 2,193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
1530,4E1C636ABEAD,1622992308071.0000,1619.0000,1810.0000,"Overall, individuals should not use cell phones while driving.\n\nThey cause distraction and lack of focus.\n\nMany lives could be saved if people weren't careful about their cell phone usage.",Concluding Statement,Concluding Statement 1,290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318


In [179]:
label[label.id=='4E1C636ABEAD'].text.values[0].replace('[BR]', '\n').split()[68:90]

['First,',
 'cell',
 'phones',
 'are',
 'a',
 'benefit',
 'and',
 'allows',
 'everyone',
 'to',
 'have',
 'access',
 'to',
 'a',
 'telephone',
 'at',
 'all',
 'times',
 'of',
 'the',
 'day.',
 'Kids,']

In [180]:
vis(3512)

ic| 2049877657.py:9 in visualize_essay()- id: '39AF0292F9B3'



 +---------+------------+------------+------------------------+--------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Overall |
|---------+------------+------------+------------------------+--------+-----------|
|  0.8333 |          1 |          1 |                      1 |      1 |    0.9667 |
+---------+------------+------------+------------------------+--------+-----------+


In [181]:
ori[ori.id=='BE01ACCDF251']

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
11821,BE01ACCDF251,1622067810486.0000,48.0000,111.0000,the advantages of limiting car usage are going out for controll,Position,Position 1,7 8 9 10 11 12 13 14 15 16 17
11822,BE01ACCDF251,1622067825243.0000,120.0000,689.0000,"the article indacties 70 percemt of Vauban's familes do not own cars, and 57 percant sold a car to move here. Also the Diesels make up 67 percent of vehicles in france , compared to a 5.3 percant average of the diesel engimes in the rest of western Europe , according to reuters. Still however indicates The goal is to promote alternative transportation and reduce smog and the Violators faced $25 fines. The Article also states As April 2013, the number of miles driven per person was nearly 9 percent below th peak and equal to where the country was in januart 1995.",Evidence,Evidence 1,19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
11823,BE01ACCDF251,1622067791674.0000,703.0000,1406.0000,"Fellow citizens about the advantages of limiting car usage are going out of controll because in the article Ingerman Suburb,Life Goes On Without Cars states Automoblies are the linchpin of suburbs, where middle- class familes From chicago to shanghai tens to make their homes and that , experts say, is a huge impediment to current efforts to drastically reduce greenhouse gas emissions from tailpies for the passenger car are responsibe for 12 percent of greenhouse gas emissions in europe and up to 50 percent in sime car intensive areas in the united states. But in paris bans driving dur to smog takes about while exceptions were made for plug in cars,hybrids, and carrying three or more passengers.",Evidence,Evidence 2,124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240
11824,BE01ACCDF251,1622067782316.0000,1422.0000,1927.0000,",fellow citizens about the advantages of limiting car usage are going out of control because it incates President obamas ambitions goals to curb the united staes greenhouse gas emissons, unveiled lastweek, will get a fortuitios assist from an incipient shoft in american behavior also recent studies suggest that americans are buying fewer cars, driving less and getting fewer licenses as each years goes by. the days without cars is part of an improvement campaign that began in bogotoa in the mid 1990s.",Evidence,Evidence 3,243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
11825,BE01ACCDF251,1622067763915.0000,1936.0000,2385.0000,",Fellow citizens about the advantages of limiting car usage are going out of control because parks and spotys center also have bloomed throughout the city, uneven , pitted sidewalk have been replaced by broad, smoth sidewalks, rush hours restrictions have dramatically cut traffic and new testaurants and upscale shopping districts have cropped up and it counties that cold nights and warm days caused the warmer layer of air to trap car emissions.",Evidence,Evidence 4,325 326 327 328 329 330 331 332 333 334 335 33

In [182]:
vis('BE01ACCDF251')

ic| 2049877657.py:9 in visualize_essay()- id: 'BE01ACCDF251'



 +------------+------------+------------------------+-----------+
|   Evidence |   Position |   Concluding Statement |   Overall |
|------------+------------+------------------------+-----------|
|          1 |          0 |                      0 |    0.3333 |
+------------+------------+------------------------+-----------+


In [183]:
label[label.id=='BE01ACCDF251'].text.values[0].replace('[BR]', '\n').split()[243:325]

['citizens',
 'about',
 'the',
 'advantages',
 'of',
 'limiting',
 'car',
 'usage',
 'are',
 'going',
 'out',
 'of',
 'control',
 'because',
 'it',
 'incates',
 'President',
 'obamas',
 'ambitions',
 'goals',
 'to',
 'curb',
 'the',
 'united',
 'staes',
 'greenhouse',
 'gas',
 'emissons,',
 'unveiled',
 'lastweek,',
 'will',
 'get',
 'a',
 'fortuitios',
 'assist',
 'from',
 'an',
 'incipient',
 'shoft',
 'in',
 'american',
 'behavior',
 'also',
 'recent',
 'studies',
 'suggest',
 'that',
 'americans',
 'are',
 'buying',
 'fewer',
 'cars,',
 'driving',
 'less',
 'and',
 'getting',
 'fewer',
 'licenses',
 'as',
 'each',
 'years',
 'goes',
 'by.',
 'the',
 'days',
 'without',
 'cars',
 'is',
 'part',
 'of',
 'an',
 'improvement',
 'campaign',
 'that',
 'began',
 'in',
 'bogotoa',
 'in',
 'the',
 'mid',
 '1990s.',
 'However,Fellow']

In [184]:
label[label.id=='BE01ACCDF251'].text.values[0].replace('[BR]', '\n').split()[242]

'Reason,fellow'

In [185]:
vis(1234)

ic| 2049877657.py:9 in visualize_essay()- id: '143881605B57'



 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|  0.8571 |          1 |          1 |                      1 |      1 |              1 |          1 |    0.9796 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


In [186]:
vis2(0)

ic| 2987334729.py:9 in visualize_ensembles()- id: '0000D23A521A'


gt


0 ensemble

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.4 |          0 |                      1 |      0 |              1 |          1 |    0.4857 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |     0.6667 |          0 |                      1 |      0 |              1 |          1 |    0.5238 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |     0.6667 |          0 |                      1 |      0 |              1 |          0 |     0.381 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.4 |          1 |                      1 |      0 |              1 |          1 |    0.6286 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |     0.6667 |          0 |                      1 |      0 |              1 |          1 |    0.5238 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |     0.6667 |          0 |                      1 |      0 |              1 |          1 |    0.5238 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.4 |          0 |                      1 |      0 |              1 |          0 |    0.3429 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.4857142857142857
    max(scores[1:]): 0.6285714285714287


In [197]:
infos[0][3]['pred']

array([[ 1.01218727, -1.16191953, -0.60575111, ...,  6.3594617 ,
        -2.098906  , -2.98096919],
       [ 1.10399456, -1.27507787, -0.62904102, ...,  6.57348588,
        -2.83009207, -3.83874001],
       [ 0.93765233, -1.21619519, -0.53054504, ...,  6.36739285,
        -2.6697502 , -3.84293842],
       ...,
       [-0.1664588 , -0.9396282 , -1.03424475, ..., -1.73835554,
        -3.46082117, -2.7248766 ],
       [-0.12204574, -0.96214861, -1.0979141 , ..., -1.74036727,
        -3.4028101 , -2.63248016],
       [-0.18810429, -0.89958203, -1.10134311, ..., -1.78325618,
        -3.24421839, -2.59012353]])

In [198]:
infos[0][3]['preds']

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [205]:
list(zip(label[label.id == '001552828BD0'].text.values[0].replace('[BR]', '\n').split(), infos[0][3]['probs'][:,5], infos[0][3]['probs'][:,2], infos[0][3]['start_probs'][:,1]))

[('Would', 0.9821572954037806, 0.001835998937870558, 0.4215315801459862),
 ('you', 0.9851095222511095, 0.0014357501748621057, 6.981155127517068e-05),
 ('be', 0.9825750435200522, 0.001821007998155811, 2.7816248671707997e-05),
 ('able', 0.9813477644320883, 0.002012941316171091, 2.6594550526910785e-05),
 ('to', 0.9813465254477649, 0.0020406702012259145, 1.9745680209726237e-05),
 ('give', 0.978102602532364, 0.0023047777825645193, 4.511768450907077e-05),
 ('your', 0.9839422164027901, 0.0020683706934423664, 1.6984782390675937e-05),
 ('car', 0.980665677708753, 0.0026109459076927317, 2.6411538167400677e-05),
 ('up?', 0.9789300274336493, 0.002508183278426261, 3.3492783446177206e-05),
 ('Having', 0.9785863525461393, 0.0029030348143017958, 0.013935576202929158),
 ('to', 0.9773388248580523, 0.0033948750499186793, 7.617376882157218e-05),
 ('ride', 0.9784263591066991, 0.003716862832501955, 7.236695019425008e-05),
 ('with', 0.9779949533205542, 0.00351158136557796, 2.1734102119322495e-05),
 ('stranger

In [187]:
vis2(3)

ic| 2987334729.py:9 in visualize_ensembles()- id: '001552828BD0'


gt


0 ensemble

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |     0.2857 |          1 |      0 |    0.3214 |
+---------+------------+------------+--------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |     0.2857 |          1 |      0 |    0.3214 |
+---------+------------+------------+--------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |        0.5 |          1 |      1 |     0.625 |
+---------+------------+------------+--------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+------------------------+--------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Overall |
|---------+------------+------------+------------------------+--------+-----------|
|       0 |        0.5 |          0 |                      0 |      1 |       0.3 |
+---------+------------+------------+------------------------+--------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |       0.25 |          1 | 0.6667 |    0.4792 |
+---------+------------+------------+--------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |       0.25 |          0 |      1 |    0.3125 |
+---------+------------+------------+--------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|     0.4 |          1 |          1 |      1 |      0.85 |
+---------+------------+------------+--------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.3214285714285714
    max(scores[1:]): 0.85


In [188]:
vis2(4)

ic| 2987334729.py:9 in visualize_ensembles()- id: '0016926B079C'


gt


0 ensemble

 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|  0.6667 |          1 |          1 |    0.8889 |
+---------+------------+------------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|  0.6154 |          1 |          1 |    0.8718 |
+---------+------------+------------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|  0.7692 |          1 |          1 |    0.9231 |
+---------+------------+------------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|     0.2 |          1 |          1 |    0.7333 |
+---------+------------+------------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|  0.6154 |          1 |          1 |    0.8718 |
+---------+------------+------------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|  0.7692 |          1 |          1 |    0.9231 |
+---------+------------+------------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+-----------+
|   Claim |   Evidence |   Position |   Overall |
|---------+------------+------------+-----------|
|  0.5455 |          1 |          1 |    0.8485 |
+---------+------------+------------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.8888888888888888
    max(scores[1:]): 0.923076923076923


In [189]:
vis2(5)

ic| 2987334729.py:9 in visualize_ensembles()- id: '0019E4D09427'


gt


0 ensemble

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|       1 |          1 |     0.6667 |                      0 |    0.6667 |
+---------+------------+------------+------------------------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+------------------------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+----------------+------------+-----------|
|       1 |          1 |     0.6667 |                      0 |              0 |          0 |    0.4444 |
+---------+------------+------------+------------------------+----------------+------------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+------------------------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+----------------+------------+-----------|
|       1 |          1 |     0.6667 |                      0 |              0 |          0 |    0.4444 |
+---------+------------+------------+------------------------+----------------+------------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|       1 |          1 |     0.6667 |                      0 |    0.6667 |
+---------+------------+------------+------------------------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|       1 |          1 |     0.6667 |                      0 |    0.6667 |
+---------+------------+------------+------------------------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|       1 |          1 |          0 |                      1 |      0.75 |
+---------+------------+------------+------------------------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+------------------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+------------+-----------|
|       1 |          1 |     0.6667 |                      0 |          0 |    0.5333 |
+---------+------------+------------+------------------------+------------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.6666666666666666
    max(scores[1:]): 0.9166666666666666


In [190]:
vis2(10)

ic| 2987334729.py:9 in visualize_ensembles()- id: '00299B378633'


gt


0 ensemble

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|  0.2857 |        0.5 |          1 |                      1 |    0.6964 |
+---------+------------+------------+------------------------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+------------------------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+----------------+------------+-----------|
|     0.5 |     0.5714 |          1 |                 0.6667 |              0 |          0 |    0.4563 |
+---------+------------+------------+------------------------+----------------+------------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+------------------------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+----------------+------------+-----------|
|     0.5 |     0.4444 |          1 |                      1 |              0 |          0 |    0.4907 |
+---------+------------+------------+------------------------+----------------+------------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|  0.2857 |          0 |          1 |                      1 |    0.5714 |
+---------+------------+------------+------------------------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+------------------------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+----------------+------------+-----------|
|  0.2857 |     0.2857 |          1 |                      1 |              0 |          0 |    0.4286 |
+---------+------------+------------+------------------------+----------------+------------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|  0.5714 |        0.5 |          1 |                      1 |    0.7679 |
+---------+------------+------------+------------------------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+------------------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Overall |
|---------+------------+------------+------------------------+-----------|
|  0.3333 |     0.6667 |          1 |                      1 |      0.75 |
+---------+------------+------------+------------------------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.6964285714285714
    max(scores[1:]): 0.9


In [191]:
vis(3)

ic| 2049877657.py:9 in visualize_essay()- id: '001552828BD0'



 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       0 |     0.2857 |          1 |      0 |    0.3214 |
+---------+------------+------------+--------+-----------+


In [192]:
vis2(11)

ic| 2987334729.py:9 in visualize_ensembles()- id: '0029F4D19C3F'


gt


0 ensemble

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|     0.4 |          0 |          0 |                      1 |      1 |              0 |          0 |    0.3429 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.5 |          0 |                      1 |      1 |              0 |          0 |    0.3571 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|     0.4 |          0 |          0 |                      1 |      1 |              1 |          0 |    0.4857 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|       0 |        0.4 |          0 |                      1 |      1 |         0.6667 |     0.6667 |    0.5333 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|  0.3333 |          0 |          0 |                      1 |      1 |              0 |          0 |    0.3333 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|     0.5 |          0 |          0 |                      0 |      1 |         0.6667 |     0.6667 |    0.4048 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+------------------------+--------+----------------+------------+-----------+
|   Claim |   Evidence |   Position |   Concluding Statement |   Lead |   Counterclaim |   Rebuttal |   Overall |
|---------+------------+------------+------------------------+--------+----------------+------------+-----------|
|     0.4 |          0 |          0 |                      1 |      1 |              0 |          0 |    0.3429 |
+---------+------------+------------+------------------------+--------+----------------+------------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.34285714285714286
    max(scores[1:]): 0.6428571428571429


In [259]:
vis2(844)

ic| 2987334729.py:9 in visualize_ensembles()- id: '0E0618CA73CD'


gt


0 ensemble

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|     0.5 |          1 |          0 |      0 |     0.375 |
+---------+------------+------------+--------+-----------+


1 deberta-v3.start.len1024.rnn_bi

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|     0.8 |          1 |          0 |      0 |      0.45 |
+---------+------------+------------+--------+-----------+


2 deberta-v3.end.len1024.seq_encoder-0

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|  0.6667 |          1 |          0 |      0 |    0.4167 |
+---------+------------+------------+--------+-----------+


...

13 longformer.start.len1536

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|     0.4 |          1 |          0 |      0 |      0.35 |
+---------+------------+------------+--------+-----------+


14 deberta-v3.start.len1536

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|     0.5 |        0.8 |          1 |      0 |     0.575 |
+---------+------------+------------+--------+-----------+


15 roberta.start.nwemb-0

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|     0.8 |          1 |          1 |      0 |       0.7 |
+---------+------------+------------+--------+-----------+


16 deberta-v3.se2

 +---------+------------+------------+--------+-----------+
|   Claim |   Evidence |   Position |   Lead |   Overall |
|---------+------------+------------+--------+-----------|
|       1 |          1 |          1 |      1 |         1 |
+---------+------------+------------+--------+-----------+


ic| 2987334729.py:32 in visualize_ensembles()
    scores[0]: 0.375
    max(scores[1:]): 1.0


In [243]:
infos[0][831]['preds']

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [246]:
infos[0][831]['probs'][:,3]

array([9.63543723e-01, 9.57661941e-01, 9.52865800e-01, 9.55654080e-01,
       9.58738645e-01, 9.54421708e-01, 9.59464521e-01, 9.56130129e-01,
       9.56353938e-01, 9.57028489e-01, 9.57220674e-01, 9.56246536e-01,
       9.54078774e-01, 9.54892192e-01, 9.58352396e-01, 9.57074438e-01,
       9.53824378e-01, 9.55842422e-01, 7.33903970e-01, 6.80006981e-01,
       6.80172464e-01, 6.81355990e-01, 6.85696689e-01, 6.86647785e-01,
       6.87976235e-01, 7.08106726e-01, 7.07011289e-01, 7.12189535e-01,
       7.13183761e-01, 7.10329490e-01, 7.15323856e-01, 7.18672992e-01,
       7.13381046e-01, 7.13140316e-01, 7.16431766e-01, 1.67032480e-02,
       1.58281320e-02, 1.30067455e-02, 1.30211507e-02, 1.26883994e-02,
       1.34665139e-02, 1.31869856e-02, 1.43376445e-02, 1.41990577e-02,
       1.57255304e-02, 3.16795865e-03, 4.17176139e-03, 4.13796855e-03,
       4.02452043e-03, 4.40254840e-03, 4.19150343e-03, 4.42101015e-03,
       4.57750080e-03, 4.22555122e-03, 4.09607938e-03, 3.89678497e-03,
      

In [247]:
infos[0][831]['start_probs'][:,1]

array([9.23779544e-01, 3.23726428e-04, 1.17664809e-04, 2.83283935e-03,
       4.35473030e-05, 6.52056314e-05, 6.33978758e-05, 5.63399934e-05,
       7.92203510e-05, 4.52480316e-05, 6.26673692e-05, 4.52694749e-05,
       2.18068991e-05, 3.36660237e-05, 4.22445754e-05, 5.39418049e-05,
       2.33995459e-05, 9.56002018e-05, 2.95084375e-01, 6.79427390e-04,
       4.15599715e-04, 1.80320104e-02, 1.99331866e-04, 3.42253463e-04,
       2.08576880e-04, 2.07943780e-04, 1.31617091e-04, 1.90963289e-04,
       1.03789605e-04, 6.34071690e-05, 8.14674855e-05, 9.22964844e-05,
       1.91299803e-04, 6.63187931e-05, 1.39284960e-04, 9.01109128e-01,
       8.56023837e-04, 8.13093793e-04, 7.43755100e-05, 2.80489706e-04,
       5.85894157e-05, 9.41096847e-05, 4.34617286e-05, 2.20770466e-04,
       2.78541223e-04, 9.15559198e-01, 1.25422677e-03, 6.09720253e-04,
       1.36383360e-04, 2.03217320e-04, 2.21754488e-04, 1.41934518e-04,
       5.62428042e-05, 5.97956805e-04, 7.58462997e-05, 8.67065214e-05,
      